In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

from tensorflow import keras
import os
import re

W0427 16:39:53.853027 140535831156608 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
import shutil

drive_path = "Clean REALEC dumps/" #@param {type:"string"}
filename = "realec_110319_2315.tar.gz" #@param {type:"string"}


shutil.copy2('/content/gdrive/My Drive/'+drive_path+filename,'.')

'./realec_110319_2315.tar.gz'

In [0]:
import tarfile

tar = tarfile.open(filename)
tar.extractall()

# Dealing with corrupt strings in annotation files

##Atypical entries

Let's fetch all annotation entries into a DataFrame. Yes, I'm that crazy.

In [5]:
%%time

all_entries_path = []
all_entries_line = []


for root, dirs, files in os.walk('.'):
  for file in os.listdir(root):
    if file.endswith(".ann"):
      path = root+os.sep+file
      with open(path, 'r', encoding='utf-8') as annfile:
        lines = annfile.read().split('\n')
        for line in lines:
          if line != "":
            all_entries_path.append(path)
            all_entries_line.append(line)


all_entries = pd.DataFrame({
  'path': all_entries_path,
  'line': all_entries_line
})

all_entries.head()

CPU times: user 2.72 s, sys: 657 ms, total: 3.38 s
Wall time: 9.33 s


In [6]:
all_entries

,path,line
0,./data/MAGOLEGO/2018/Student_36_MT.ann,People now fully enjoy the benefits this age ...
1,./data/MAGOLEGO/2018/Student_36_MT.ann,"What people often fail to notice, however, is..."
2,./data/MAGOLEGO/2018/Student_36_MT.ann,Perhaps it is now the right time to conceive ...
3,./data/MAGOLEGO/2018/Student_36_MT.ann,The debate on the topic is quite heated and m...
4,./data/MAGOLEGO/2018/Student_36_MT.ann,The critics claim that huge market shares all...
5,./data/MAGOLEGO/2018/Student_36_MT.ann,"Secondly, the critics are often concerned wit..."
6,./data/MAGOLEGO/2018/Student_36_MT.ann,"Finally, many of the speakers, for example, J..."
7,./data/MAGOLEGO/2018/Student_36_MT.ann,"Ultimately, speakers from this side call for ..."
8,./data/MAGOLEGO/2018/Student_36_MT.ann,The other side of the debate stands against t...
9,./data/MAGOLEGO/2018/Student_36_MT.ann,There are several lines of reasoning the spea...


Back to work. Now let's figure out it there are any entries which don't follow the regular pattern.

In [7]:
%%time

atypical_entries = pd.DataFrame({
    'path': [],
    'line': []
})

for root, dirs, files in os.walk('.'):
  for file in os.listdir(root):
    if file.endswith(".ann"):
      path = root+os.sep+file
      with open(path, 'r', encoding='utf-8') as annfile:
        lines = annfile.read().split('\n')
        for line in lines:
          if line != "":
            if not re.search(r'^.[0-9]+\t', line):
              atypical_entries.loc[len(atypical_entries)] = [path, line]

CPU times: user 5.47 s, sys: 162 ms, total: 5.63 s
Wall time: 5.69 s


In [8]:
atypical_entries

,path,line
0,./data/MAGOLEGO/2018/Student_36_MT.ann,People now fully enjoy the benefits this age ...
1,./data/MAGOLEGO/2018/Student_36_MT.ann,"What people often fail to notice, however, is..."
2,./data/MAGOLEGO/2018/Student_36_MT.ann,Perhaps it is now the right time to conceive ...
3,./data/MAGOLEGO/2018/Student_36_MT.ann,The debate on the topic is quite heated and m...
4,./data/MAGOLEGO/2018/Student_36_MT.ann,The critics claim that huge market shares all...
5,./data/MAGOLEGO/2018/Student_36_MT.ann,"Secondly, the critics are often concerned wit..."
6,./data/MAGOLEGO/2018/Student_36_MT.ann,"Finally, many of the speakers, for example, J..."
7,./data/MAGOLEGO/2018/Student_36_MT.ann,"Ultimately, speakers from this side call for ..."
8,./data/MAGOLEGO/2018/Student_36_MT.ann,The other side of the debate stands against t...
9,./data/MAGOLEGO/2018/Student_36_MT.ann,There are several lines of reasoning the spea...


Let's just remove those from our general set so we don't have to worry about them afterwards. We'll also test for the time such an operation takes here.

In [9]:
%%time

all_entries = pd.concat([all_entries, atypical_entries]).drop_duplicates(keep=False)

CPU times: user 4.54 s, sys: 273 ms, total: 4.82 s
Wall time: 4.83 s


Let's check if ```all_entries``` size has changed:


In [10]:
all_entries

,path,line
15,./data/exam/exam2017_3/DOv_7_1.ann,T1\tPrepositions 60 64\tfrom
16,./data/exam/exam2017_3/DOv_7_1.ann,#1\tAnnotatorNotes T1\tof
17,./data/exam/exam2017_3/DOv_7_1.ann,T2\tRedundant_comp 79 85\tgroups
18,./data/exam/exam2017_3/DOv_7_1.ann,A1\tDelete T2
19,./data/exam/exam2017_3/DOv_7_1.ann,T3\tNoun_number 75 78\tage
20,./data/exam/exam2017_3/DOv_7_1.ann,#2\tAnnotatorNotes T3\tages
21,./data/exam/exam2017_3/DOv_7_1.ann,R1\tDependent_change Arg1:T2 Arg2:T3\t
22,./data/exam/exam2017_3/DOv_7_1.ann,R2\tDependent_change Arg1:T3 Arg2:T1\t
23,./data/exam/exam2017_3/DOv_7_1.ann,T4\tSpelling 177 183\tnetwor
24,./data/exam/exam2017_3/DOv_7_1.ann,#3\tAnnotatorNotes T4\tnetwork


Let's look which string prefixes we have:

In [32]:
linestarts = []

for line in list(all_entries["line"]):
  val = re.search(r'^(.*?)[0-9]', line).group(1)
  if val not in linestarts:
    linestarts.append(val)

linestarts

['T', '#', 'A', 'R']

What are the `R` tags?

In [29]:
all_entries.loc[all_entries.line.str.contains('^R[0-9]'), :][:10]

,path,line
21,./data/exam/exam2017_3/DOv_7_1.ann,R1\tDependent_change Arg1:T2 Arg2:T3\t
22,./data/exam/exam2017_3/DOv_7_1.ann,R2\tDependent_change Arg1:T3 Arg2:T1\t
1335,./data/exam/exam2017_3/AGo_1_2.ann,R1\tDependent_change Arg1:T16 Arg2:T17\t
1434,./data/exam/exam2017_3/AGo_1_2.ann,R2\tDependent_change Arg1:T65 Arg2:T66\t
7233,./data/exam/best_works/9_2.ann,R1\tDependent_change Arg1:T6 Arg2:T7\t
8505,./data/exam/best_works/33_2.ann,R1\tParallel_construction Arg1:T12 Arg2:T13\t
8508,./data/exam/best_works/33_2.ann,R2\tParallel_construction Arg1:T12 Arg2:T14\t
9811,./data/exam/best_works/17_2.ann,R1\tDependent_change Arg1:T21 Arg2:T22\t
15375,./data/exam/best_works/8_2.ann,R1\tDependent_change Arg1:T23 Arg2:T24\t
16011,./data/exam/best_works/19_2.ann,R1\tDependent_change Arg1:T20 Arg2:T21\t


So, what type of `R` tags causes do we have?

In [38]:
r_types = []

for line in list(all_entries["line"]):
  query = re.search(r'^R[0-9]+\t(.*?) ', line)
  if query:
    val = query.group(1)
  if val not in r_types:
    r_types.append(val)

r_types

['Dependent_change', 'Parallel_construction', 'Discourse']

What about the `A` non-delete ones?

In [39]:
a_types = []

for line in list(all_entries["line"]):
  query = re.search(r'^A[0-9]+\t(.*?) ', line)
  if query:
    val = query.group(1)
  if val not in a_types:
    a_types.append(val)

a_types

['Dependent_change',
 'Delete',
 'Cause',
 'Addition',
 'Weight-language',
 'Weight-understanding']

In [45]:
all_entries.loc[all_entries.line.str.contains('^A[0-9]+\tAddition'), :][:10]

,path,line
164508,./data/exam/exam2017_5_1/EGe_15_2.ann,A1\tAddition T11
395030,./data/exam/exam2017/EGe_136_2.ann,A7\tAddition T6
570633,./data/exam/exam2017_5_2/EGe_160_1.ann,A3\tAddition T21
581335,./data/exam/exam2017_5_2/EGe_127_1.ann,A4\tAddition T24
581336,./data/exam/exam2017_5_2/EGe_127_1.ann,A5\tAddition T31
581337,./data/exam/exam2017_5_2/EGe_127_1.ann,A7\tAddition T40
583735,./data/exam/exam2017_5_2/EGe_145_1.ann,A4\tAddition T7
607978,./data/exam/exam2017_5_2/EGe_147_2.ann,A4\tAddition T1
666123,./data/exam/exam2015/KT_1_1.ann,A1\tAddition T194
691451,./data/exam/exam2016/MTsy_9_1.ann,A5\tAddition T27


What about `#`?

In [56]:
DASH_types = []

for line in list(all_entries["line"]):
  query = re.search(r'^\#[0-9]+\t(.*?) ', line)
  if query:
    val = query.group(1)
  if val not in DASH_types:
    DASH_types.append(val)

DASH_types

['AnnotatorNotes']

Based on these observations, I decided to further work with the `Delete` entries from all non-`A` tags and further narrowed my field of error tags to the following ones:

In [48]:
Allowed_types = ["Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Tense_choice", "Tense_form", "Voice", "Modals", "Verb_pattern", "Intransitive", "Transitive", "Reflexive_verb", "Presentation", "Ambitransitive", "Two_in_a_row", "Verb_Inf", "Verb_Gerund", "Verb_Inf_Gerund", "Verb_Bare_Inf", "Verb_object_bare", "Restoration_alter", "Verb_part", "Get_part", "Complex_obj", "Verbal_idiom", "Prepositional_verb", "Dative", "Followed_by_a_clause", "that_clause", "if_whether_clause", "that_subj_clause", "it_conj_clause", "Participial_constr", "Infinitive_constr", "Gerund_phrase", "Nouns", "Countable_uncountable", "Prepositional_noun", "Possessive", "Noun_attribute", "Noun_inf", "Noun_number", "Prepositions", "Conjunctions", "Adjectives", "Comparative_adj", "Superlative_adj", "Prepositional_adjective", "Adj_as_collective", "Adverbs", "Comparative_adv", "Superlative_adv", "Prepositional_adv", "Numerals", "Pronouns", "Agreement_errors", "Word_order", "Standard", "Emphatic", "Cleft", "Interrogative", "Abs_comp_clause", "Exclamation", "Title_structure", "Note_structure", "Conditionals", "Attributes", "Relative_clause", "Defining", "Non_defining", "Coordinate", "Attr_participial", "Lack_par_constr", "Negation", "Comparative_constr", "Numerical", "Confusion_of_structures", "Vocabulary", "Word_choice", "lex_item_choice", "Often_confused", "lex_part_choice", "Absence_comp_colloc", "Redundant", "Derivation", "Formational_affixes", "Suffix", "Prefix", "Category_confusion", "Compound_word", "Discourse", "Ref_device", "Coherence", "Linking_device", "Inappropriate_register", "Absence_comp_sent", "Redundant_comp", "Absence_explanation"]

Allowed_types

['Punctuation',
 'Spelling',
 'Capitalisation',
 'Grammar',
 'Determiners',
 'Articles',
 'Quantifiers',
 'Verbs',
 'Tense',
 'Tense_choice',
 'Tense_form',
 'Voice',
 'Modals',
 'Verb_pattern',
 'Intransitive',
 'Transitive',
 'Reflexive_verb',
 'Presentation',
 'Ambitransitive',
 'Two_in_a_row',
 'Verb_Inf',
 'Verb_Gerund',
 'Verb_Inf_Gerund',
 'Verb_Bare_Inf',
 'Verb_object_bare',
 'Restoration_alter',
 'Verb_part',
 'Get_part',
 'Complex_obj',
 'Verbal_idiom',
 'Prepositional_verb',
 'Dative',
 'Followed_by_a_clause',
 'that_clause',
 'if_whether_clause',
 'that_subj_clause',
 'it_conj_clause',
 'Participial_constr',
 'Infinitive_constr',
 'Gerund_phrase',
 'Nouns',
 'Countable_uncountable',
 'Prepositional_noun',
 'Possessive',
 'Noun_attribute',
 'Noun_inf',
 'Noun_number',
 'Prepositions',
 'Conjunctions',
 'Adjectives',
 'Comparative_adj',
 'Superlative_adj',
 'Prepositional_adjective',
 'Adj_as_collective',
 'Adverbs',
 'Comparative_adv',
 'Superlative_adv',
 'Prepositional_ad

# Further processing

Let's separate all `A Delete`, `#` and `T`-type tags from the rest:

In [51]:
ADelete_entries = all_entries.loc[all_entries.line.str.contains('^A[0-9]+\tDelete'), :]

ADelete_entries.head()

,path,line
18,./data/exam/exam2017_3/DOv_7_1.ann,A1\tDelete T2
206,./data/exam/exam2017_3/DOv_4_1.ann,A1\tDelete T9
222,./data/exam/exam2017_3/DOv_4_1.ann,A2\tDelete T17
1210,./data/exam/exam2017_3/DOv_9_1.ann,A1\tDelete T5
1216,./data/exam/exam2017_3/DOv_9_1.ann,A2\tDelete T8


In [53]:
DASH_entries = all_entries.loc[all_entries.line.str.contains('^\#[0-9]'), :]

DASH_entries.head()

,path,line
16,./data/exam/exam2017_3/DOv_7_1.ann,#1\tAnnotatorNotes T1\tof
20,./data/exam/exam2017_3/DOv_7_1.ann,#2\tAnnotatorNotes T3\tages
24,./data/exam/exam2017_3/DOv_7_1.ann,#3\tAnnotatorNotes T4\tnetwork
26,./data/exam/exam2017_3/DOv_7_1.ann,#4\tAnnotatorNotes T5\tall
28,./data/exam/exam2017_3/DOv_7_1.ann,#5\tAnnotatorNotes T6\tin


In [130]:
T_entries = all_entries.loc[all_entries.line.str.contains('^T[0-9]'), :]

T_entries.head()

,path,line
15,./data/exam/exam2017_3/DOv_7_1.ann,T1\tPrepositions 60 64\tfrom
17,./data/exam/exam2017_3/DOv_7_1.ann,T2\tRedundant_comp 79 85\tgroups
19,./data/exam/exam2017_3/DOv_7_1.ann,T3\tNoun_number 75 78\tage
23,./data/exam/exam2017_3/DOv_7_1.ann,T4\tSpelling 177 183\tnetwor
25,./data/exam/exam2017_3/DOv_7_1.ann,T5\tQuantifiers 243 246\tany


`A`-entries should follow the pattern `A[any number]\tDelete␣T[any number]`. Let's check if we have any outliers!

In [62]:
ADelete_invalid = ADelete_entries.loc[~ADelete_entries.line.str.contains('^A[0-9]+\tDelete T[0-9]+$'), :]

ADelete_invalid

,path,line


No invalid entries here. Moving to `#`, which should be `#[any number]\tAnnotatorNotes␣T[any number]\t[any text]`

In [0]:
DASH_invalid = DASH_entries.loc[~DASH_entries.line.str.contains('^#[0-9]+\tAnnotatorNotes T[0-9]+\t.*$'), :]

DASH_invalid

So there seem to be two cases: empty `AnnotatorNotes`, which we will assume to be en empty correction, and other stuff. Let's focus on the latter for now.

In [0]:
DASH_invalid_nonempty = DASH_entries.loc[~DASH_entries.line.str.contains('^#[0-9]+\tAnnotatorNotes T[0-9]+.*$'), :]

DASH_invalid_nonempty

We'll drop this ocurrence and add a trailing tabulation to the other irregular lines to unify our notation:

In [0]:
DASH_entries = pd.concat([DASH_entries, DASH_invalid_nonempty]).drop_duplicates(keep=False)

In [0]:
condition = ~DASH_entries.line.str.contains('^#[0-9]+\tAnnotatorNotes T[0-9]+\t.*$')

DASH_entries.loc[condition, 'line'] = DASH_entries.loc[condition, 'line']+'\t'

# DASH_entries.loc[condition, 'line'].apply(appendix) = DASH_entries.loc[condition, 'line'].apply(appendix)

In [100]:
DASH_new_invalid = DASH_entries.loc[~DASH_entries.line.str.contains('^#[0-9]+\tAnnotatorNotes T[0-9]+\t.*$'), :]

DASH_new_invalid

,path,line


So, this narrows us down to `T`. These should be `T[any number]\t[error type]␣[any number]␣[any number]\t[any text]`. Let's test!

In [0]:
T_invalid = T_entries.loc[~T_entries.line.str.contains('^T[0-9]+\t[\w_]+ [0-9]+ [0-9]+\t.*?$'), :]

T_invalid

Right off the bat we can see that we have some double-staked entries here. Our best take is to divide these entries and treat them as independent tokens. Let's separate and process them:

In [0]:
T_double = T_entries.loc[T_entries.line.str.contains('^T[0-9]+\t[\w_]+ [0-9]+ [0-9]+;[0-9]+ [0-9]+\t.*?$'), :]

T_double

In [0]:
DEZip = zip(list(T_double.path), list(T_double.line))
double_entries = []
for e in DEZip:
  double_entries.append([e[0], e[1]])
double_entries

In [0]:
dedoubled_entries = []
for entry in double_entries:
  e = entry[1]
  head = re.search(r'^(T[0-9]+\t[\w_]+ )[0-9]+ [0-9]+;[0-9]+ [0-9]+\t.*?$', e).group(1)
  range1 = re.search(r'^T[0-9]+\t[\w_]+ ([0-9]+ [0-9]+);[0-9]+ [0-9]+\t.*?$', e).group(1)
  range2 = re.search(r'^T[0-9]+\t[\w_]+ [0-9]+ [0-9]+;([0-9]+ [0-9]+)\t.*?$', e).group(1)
  string = re.search(r'^T[0-9]+\t[\w_]+ [0-9]+ [0-9]+;[0-9]+ [0-9]+\t(.*?)$', e).group(1)
  len1 = int(range1.split(" ")[1])-int(range1.split(" ")[0])
  str1 = string[:len1]
  str2 = string[len1+1:]
  dedoubled_entries.append([entry[0], head+range1+'\t'+str1])
  dedoubled_entries.append([entry[0], head+range2+'\t'+str2])

dedoubled_entries

In [0]:
dedoubled = pd.DataFrame({
  'path': [],
  'line': []
})

for line in dedoubled_entries:
  dedoubled.loc[len(dedoubled)] = line

dedoubled

In [0]:
T_entries = pd.concat([T_entries, T_double]).drop_duplicates(keep=False)
T_entries = pd.concat([T_entries, dedoubled])

T_entries

Now let's get rid of all the tokens we will not be including in our analysis:

In [0]:
T_invalid = T_entries.loc[~T_entries.line.str.contains('^T[0-9]+\t('+'|'.join(Allowed_types)+') [0-9]+ [0-9]+'), :]

T_invalid

These are mostly parts of speech and suggestions. To hell with them:

In [0]:
T_entries = pd.concat([T_entries, T_invalid]).drop_duplicates(keep=False)

T_entries

Finally let's deal with the entries without suggestions. Like above, let's just find them and add a trailing tabulation:

In [0]:
T_nontabbed = T_entries.loc[~T_entries.line.str.contains('^T[0-9]+\t('+'|'.join(Allowed_types)+') [0-9]+ [0-9]+\t.*?$'), :]

T_nontabbed

In [0]:
condition = ~T_entries.line.str.contains('^T[0-9]+\t('+'|'.join(Allowed_types)+') [0-9]+ [0-9]+\t.*?$')

T_entries.loc[condition, 'line'] = T_entries.loc[condition, 'line']+'\t'

In [141]:
T_new_invalid = T_entries.loc[~T_entries.line.str.contains('^T[0-9]+\t('+'|'.join(Allowed_types)+') [0-9]+ [0-9]+\t.*?$'), :]

T_new_invalid

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


,path,line
460810,./data/exam/exam2017/NMya_26_1.ann,T4\tSpelling 421 431;49 59;561 571\tpersentage...


We've still got one monster left, as it seems. Let's drop it

In [0]:
T_entries = pd.concat([T_entries, T_new_invalid]).drop_duplicates(keep=False)

Let's reset indices of all resulting dataframes here:

In [0]:
ADelete_entries = ADelete_entries.reset_index(drop=True)
DASH_entries = DASH_entries.reset_index(drop=True)
T_entries = T_entries.reset_index(drop=True)

## Uploading the results

Wow. That was epic and took a looong time. We'd better save it into a json file to accelerate future load-ups.

In [167]:
%%time

jsonname = "DASH_entries.json" #@param {type:"string"}
with open(jsonname, 'w', encoding="utf-8") as outie:
    exec("outie.write("+jsonname[:-5]+".to_json())")

CPU times: user 1.27 s, sys: 691 ms, total: 1.96 s
Wall time: 2.04 s


Whew, great! Let's upload this to Google Drive right now. 

In [163]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 19.8MB/s 


In [168]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': jsonname})
uploaded.SetContentFile(jsonname)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1txk_7dfZIgxy6wOOTSNk6v6N12-GTxRf
